<a href="https://colab.research.google.com/github/zhangxs131/Tensorflow_learning/blob/main/imdb_with_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##tensorflow hub对imdb文本分类

这里没用使用keras，而是直接使用tensorflow-hub进行文本分类,这里读取数据集直接为可以送入模型的tensor类型

In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets
!pip install tfds-nightly

     |████████████████████████████████| 4.3 MB 4.3 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 


In [4]:
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Eager Mode',tf.executing_eagerly())
print('GPU is ','available' if tf.config.experimental.list_physical_devices('GPU') else 'NOT avaible')


Eager Mode True
GPU is  available


In [7]:
#dataset
train_data,valid_data,test_data=tfds.load(
    name='imdb_reviews',
    split=('train[:60%]','train[60%:]','test'),
    as_supervised=True
)

train_example_batch,train_labels_batch=next(iter(train_data.batch(10)))
print(train_example_batch.shape)
print(train_labels_batch.shape)


(10,)
(10,)


In [8]:
print(train_example_batch)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

##从tfhub上加载embedding模型

In [10]:
embedding="https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer=hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)
r=hub_layer(train_example_batch[:3])
print(r.shape)
print(r)

(3, 50)
tf.Tensor(
[[ 0.5423194  -0.01190171  0.06337537  0.0686297  -0.16776839 -0.10581177
   0.168653   -0.04998823 -0.31148052  0.07910344  0.15442258  0.01488661
   0.03930155  0.19772716 -0.12215477 -0.04120982 -0.27041087 -0.21922147
   0.26517656 -0.80739075  0.25833526 -0.31004202  0.2868321   0.19433866
  -0.29036498  0.0386285  -0.78444123 -0.04793238  0.41102988 -0.36388886
  -0.58034706  0.30269453  0.36308962 -0.15227163 -0.4439151   0.19462997
   0.19528405  0.05666233  0.2890704  -0.28468323 -0.00531206  0.0571938
  -0.3201319  -0.04418665 -0.08550781 -0.55847436 -0.2333639  -0.20782956
  -0.03543065 -0.17533456]
 [ 0.56338924 -0.12339553 -0.10862677  0.7753425  -0.07667087 -0.15752274
   0.01872334 -0.08169781 -0.3521876   0.46373403 -0.08492758  0.07166861
  -0.00670818  0.12686071 -0.19326551 -0.5262643  -0.32958236  0.14394784
   0.09043556 -0.54175544  0.02468163 -0.15456744  0.68333143  0.09068333
  -0.45327246  0.23180094 -0.8615696   0.3448039   0.12838459 -0.58

In [12]:
model=tf.keras.Sequential([
  hub_layer,
  tf.keras.layers.Dense(16,activation='relu'),
  tf.keras.layers.Dense(1)                         
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history=model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=10,
    validation_data=valid_data.batch(512),
    verbose=1
)

Epoch 1/10
30/30 [==============================] - 7s 108ms/step - loss: 0.6978 - accuracy: 0.5179 - val_loss: 0.6423 - val_accuracy: 0.5392
Epoch 2/10
30/30 [==============================] - 3s 84ms/step - loss: 0.5901 - accuracy: 0.6259 - val_loss: 0.5513 - val_accuracy: 0.6785
Epoch 3/10
30/30 [==============================] - 3s 82ms/step - loss: 0.4718 - accuracy: 0.7667 - val_loss: 0.4512 - val_accuracy: 0.7819
Epoch 4/10
30/30 [==============================] - 3s 79ms/step - loss: 0.3518 - accuracy: 0.8593 - val_loss: 0.3789 - val_accuracy: 0.8373
Epoch 5/10
30/30 [==============================] - 3s 80ms/step - loss: 0.2613 - accuracy: 0.9053 - val_loss: 0.3383 - val_accuracy: 0.8546
Epoch 6/10
30/30 [==============================] - 3s 82ms/step - loss: 0.1960 - accuracy: 0.9359 - val_loss: 0.3176 - val_accuracy: 0.8622
Epoch 7/10
30/30 [==============================] - 3s 81ms/step - loss: 0.1485 - accuracy: 0.9553 - val_loss: 0.3096 - val_accuracy: 0.8632
Epoch 8/10
3

In [15]:
#evluate
result=model.evaluate(test_data.batch(512),verbose=1)

for name,value in zip(model.metrics_names,result):
  print(name,':',value)

49/49 [==============================] - 2s 31ms/step - loss: 0.3418 - accuracy: 0.8525
loss : 0.3418482542037964
accuracy : 0.8524799942970276
